In [ ]:
from rasterstats import zonal_stats
import os
import csv
import gdal
import osr

### 1. Import original tif files and divide by 10k

In [ ]:
os.chdir('C:/Users/SNMACIAS/Documents/CIMMYT/Drone_missions/eBee/AGF/q190930for/analysis/rho_leaf/')
# You have to divide the original *.tif file by 10k otherwise, div by zero error
#origfile = 'IMG_190920_160935_0000_GRE_div_by_10k.TIF'

In [ ]:
# Open the file - NOT WORKING SO FAR
fid=gdal.Open(origfile, gdal.GA_ReadOnly)
rows=fid.RasterYSize
cols=fid.RasterXSize

# Read relevant bands and store them as arrays
refl=fid.GetRasterBand(1).ReadAsArray()
del fid

# Prepare image
in_set = gdal.Open(origfile)
driver = gdal.GetDriverByName("GTiff")                     # Define driver for image creation
dType = gdal.GDT_Float32

epsg = 4326                                                # Read projection
srs = osr.SpatialReference()
srs.ImportFromEPSG(epsg)
srs = srs.ExportToWkt()
#projection = in_set.GetProjection()
geolocation = in_set.GetGeoTransform()                     # Read georeference

In [ ]:
# Divide by 10k and write file - NOT WORKING SO FAR
outArray = in_set.GetRasterBand(1).ReadAsArray()
outArray = np.divide(refl, 10000)                          # Div by 10k

img_out = 'IMG_190930_172529_0003_REG_div_by_10k.tif'
out_set = driver.Create(img_out, cols, rows, 1, dType)    # Image creation
#out_set.SetProjection(srs)                                # Assign projection
#out_set.SetGeoTransform(geolocation)                      # Assign geolocation

#out_set.GetRasterBand(1).WriteArray(outArray)             # Write file

### 2. Calculate zonal statistics

In [ ]:
# GRE, NIR, RED, REG
rastfile = 'IMG_190930_172529_0003_REG_cal.tif' # REG_cal.tif
shpfile = 'REG_172529.shp'

In [ ]:
stats0 = zonal_stats(shpfile, rastfile, stats="count min mean max median", band=1, geojson_out=True)
stats1 = zonal_stats(shpfile, rastfile, stats="count min mean max median", band=1)

### Calibration with panel reflectance

In [ ]:
fac = 0.268/stats1[0]['mean'] # 0.178:GRE, 0.369:NIR, 0.216:RED, 0.268:REG
fac

In [ ]:
stats1[0]['mean'] # stats0[0]['properties']['mean']

### Estimate leaf reflectance

In [ ]:
for i in range(len(stats0)):
    var = stats1[i]['mean']
    print(var)

In [ ]:
with open ('output.csv', 'w', newline="") as f:
    fieldnames = 'count','min', 'mean', 'max', 'median'
    cw = csv.DictWriter(f, fieldnames) #  extrasaction='ignore'
    cw.writerows(stats1)